In [1]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
from keras.models import Sequential
from tensorflow import keras
from scipy import signal
import os

In [2]:
# assign directory
directory = "drive/MyDrive/MINT_COLOR_DATA"
 
# iterate over files in
# that directory
dataset=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        ds = pd.read_csv(f)
        i=ds.loc[ds[' EXG Channel 0'] == 0].index
        if(len(i)==1):
          ds=ds.tail(-(i[0]+1))
        dataset.append(ds)

In [3]:
dataset[6]

,Sample Index,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Accel Channel 0,...,Other.3,Other.4,Other.5,Other.6,Analog Channel 0,Analog Channel 1,Analog Channel 2,Timestamp,Other.7,Timestamp (Formatted)
124,1.0,-108141.025381,-148558.321423,-70007.362367,-133586.743008,39870.102986,-81548.395043,30519.809487,-119777.075324,0.072,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:18.829
125,2.0,-108145.383971,-148560.981281,-70011.832715,-133580.886850,39867.800756,-81532.815878,30535.455708,-119772.515568,0.072,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:18.829
126,3.0,-108144.691067,-148558.611996,-70013.665559,-133580.149243,39866.906687,-81525.685671,30553.113586,-119767.799350,0.072,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:18.830
127,4.0,-108141.204195,-148559.774287,-70006.088317,-133579.299877,39875.757977,-81522.131744,30554.253525,-119760.423274,0.072,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:18.830
128,5.0,-108152.804751,-148575.196990,-70020.438137,-133601.405752,39857.183678,-81548.015064,30528.526667,-119784.227882,0.072,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:18.830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730,255.0,-109727.239278,-148882.488773,-71265.050324,-134003.893614,41112.815274,-82599.575233,32126.698747,-119317.232885,0.026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:37.058
4731,0.0,-109731.128482,-148886.959122,-71263.150425,-133986.392198,41114.223434,-82587.102960,32150.481004,-119318.998673,0.026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:37.058
4732,1.0,-109732.179014,-148884.008692,-71269.140693,-133993.656515,41107.562614,-82598.077667,32159.645219,-119323.737243,0.026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:37.058
4733,2.0,-109735.196499,-148884.142802,-71279.511902,-134011.828484,41099.560690,-82618.462457,32142.233210,-119323.938408,0.042,...,29.0,208.0,1.0,48.0,0.0,0.0,0.0,1.666404e+09,0.0,2022-10-21 18:52:37.058


In [4]:
for data in dataset:
  data.drop(data.iloc[:, 9:25], axis=1, inplace=True)
dataset[2].shape

(4508, 9)

In [5]:
i =0;
for data in dataset:
  if data.shape[0] < 4608:
    i=i+1
i

21

In [6]:
data_train=dataset[0:40]
data_test=dataset[40:52]

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler()
i=0;
for data in dataset[0:40]:
  data_train[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1
i=0
for data in dataset[40:52]:
  data_test[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1

In [8]:
import tqdm #0 is green, 1 is blue, 2 is red
n=256*5
feature_names=list(data_train[0].drop('Sample Index',axis=1).columns)
X=[]
Y=[]
indexes=[]
for data in data_train:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(3)):
    X.append(norm_data_x.iloc[i*n:i*n+n].  values)
    Y.append(i)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
print(X[119].shape)
print(Y[119])

(1280, 8)
2


In [89]:
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import adam_v2
n_steps=256*5
n_features=8
model=Sequential()
model.add(LSTM(100, activation='relu', return_sequences=False, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(Dense(1))
opt = adam_v2.Adam(lr=0.0005, decay=1e-6)

model.compile(optimizer="adam", loss='mse', metrics=['mape'])

In [90]:
history=model.fit(np.asarray(X),np.asarray(Y),batch_size=20,epochs=15)

Epoch 1/15
6/6 [==============================] - 4s 550ms/step - loss: 1.1020 - mape: 112441824.0000
Epoch 2/15
6/6 [==============================] - 3s 546ms/step - loss: 0.7619 - mape: 340363328.0000
Epoch 3/15
6/6 [==============================] - 3s 543ms/step - loss: 0.5785 - mape: 278697600.0000
Epoch 4/15
6/6 [==============================] - 4s 764ms/step - loss: 0.4530 - mape: 224813776.0000
Epoch 5/15
6/6 [==============================] - 3s 539ms/step - loss: 0.2975 - mape: 196795040.0000
Epoch 6/15
6/6 [==============================] - 3s 542ms/step - loss: 0.2206 - mape: 158136784.0000
Epoch 7/15
6/6 [==============================] - 3s 542ms/step - loss: 0.1568 - mape: 104571976.0000
Epoch 8/15
6/6 [==============================] - 3s 544ms/step - loss: 0.1335 - mape: 87697840.0000
Epoch 9/15
6/6 [==============================] - 3s 546ms/step - loss: 0.1498 - mape: 69365880.0000
Epoch 10/15
6/6 [==============================] - 3s 544ms/step - loss: 0.1422 - ma

In [62]:
n=256*5
feature_names=list(data_test[0].drop('Sample Index',axis=1).columns)
X_test=[]
Y_test=[]
indexes=[]
for data in data_test:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(3)):
    X_test.append(norm_data_x.iloc[i*n:i*n+n].  values)
    Y_test.append(i)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [91]:
predictions = model.predict(np.asarray(X_test))
predictions = pd.DataFrame(predictions).rename(columns={0: 'prediction'})
actual=pd.DataFrame(Y_test).rename(columns={0: 'actual'})
final=pd.concat([predictions,actual],axis=1)
final['mae']=abs(final['prediction']-final['actual'])
final

2/2 [==============================] - 0s 81ms/step


,prediction,actual,mae
0,0.352190,0,0.352190
1,1.097873,1,0.097873
2,2.273346,2,0.273346
3,0.343127,0,0.343127
4,0.817668,1,0.182332
5,2.234143,2,0.234143
6,0.845951,0,0.845951
7,1.091040,1,0.091040
8,1.471939,2,0.528061
9,0.277475,0,0.277475


In [92]:
import pickle
filename = 'finalized_color_model.sav'
pickle.dump(model, open(filename, 'wb'))